In [1]:
import pandas as pd
import glob
import re
from tqdm import tqdm
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
path = r'20200705v1/full/processed' # use your path
all_files = glob.glob(path + "/body_txt_*.feather")

In [3]:
#nltk.download("punkt")
#print("-----------------------------")
#nltk.download("wordnet")
#print("-----------------------------")
#nltk.download("stopwords")

-----------------------------
-----------------------------


[nltk_data] Downloading package punkt to /Users/udayts/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/udayts/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/udayts/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def process_textfile(path):
 
    file_index = re.findall('\d+', path)[-1]
    
    df = pd.read_feather(path)
    
    df_abstract = df.drop(columns=["body_text"])
    df_bodytext = df.drop(columns=['abstract_x'])
    df_bodytext.dropna(inplace=True)
    df_bodytext=df_bodytext[df_bodytext.body_text!='']
    df_bodytext.reset_index(inplace=True, drop=True)
    
    '''
    replace symbols with '' 
    '''
    
    p_signs = list("><?:;!.,&~+=-)/%(_")
    df_abstract["abstract_x_p_parsed0"]=df_abstract["abstract_x"]
    df_bodytext["bodytext_p_parsed0"]=df_bodytext["body_text"]
    for sign in p_signs:
        df_abstract["abstract_x_p_parsed0"]=df_abstract["abstract_x_p_parsed0"].str.replace(sign, '')
        df_bodytext["bodytext_p_parsed0"]=df_bodytext["bodytext_p_parsed0"].str.replace(sign, '')
    
    '''
    replace 's pronouns with '' 
    '''    
    
    df_abstract["abstract_x_p_parsed1"] = df_abstract["abstract_x_p_parsed0"].str.replace("'s", "")
    df_bodytext["bodytext_p_parsed1"] = df_bodytext["bodytext_p_parsed0"].str.replace("'s", "")
    
    '''Lemmatization'''
    
    wordnet_lemmatizer = WordNetLemmatizer()
    x_rows = len(df_abstract)
    y_rows = len(df_bodytext)
    lemma_abstract_textlist = []
    lemma_bodytext_textlist = []
    for row in range(0,x_rows):
        lemma_abstract_list = []
        text_abstract = df_abstract.loc[row]["abstract_x_p_parsed1"]
        text_abstract_words = text_abstract.split(" ")
        for abstract_word in text_abstract_words:
            lemma_abstract_list.append(wordnet_lemmatizer.lemmatize(abstract_word, pos="v"))
        lemma_abstract_text = " ".join(lemma_abstract_list)
        lemma_abstract_textlist.append(lemma_abstract_text)
    for row in range(0,y_rows):
        lemma_bodytext_list = []
        text_bodytext = df_bodytext.loc[row]["bodytext_p_parsed1"]
        text_bodytext_words = text_bodytext.split(" ")
        for bodytext_word in text_bodytext_words:
            lemma_bodytext_list.append(wordnet_lemmatizer.lemmatize(bodytext_word, pos="v"))
        lemma_bodytext_text = " ".join(lemma_bodytext_list)
        lemma_bodytext_textlist.append(lemma_bodytext_text)
    df_abstract["abstract_x_p_parsed2"] = lemma_abstract_textlist
    df_bodytext["bodytext_p_parsed2"] = lemma_bodytext_textlist    
    
    stop_words = list(stopwords.words('english'))
    df_abstract["abstract_x_p_parsed3"] = df_abstract["abstract_x_p_parsed2"]
    df_bodytext["bodytext_p_parsed3"] = df_bodytext["bodytext_p_parsed2"]
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df_abstract["abstract_x_p_parsed3"] = df_abstract["abstract_x_p_parsed3"].str.replace(regex_stopword, "")
        df_bodytext["bodytext_p_parsed3"] = df_bodytext["bodytext_p_parsed3"].str.replace(regex_stopword, "")
    
    list_col_abstract = ['paper_id', 'abstract_x', 'mag_field_of_study', 'abstract_x_p_parsed3']
    list_col_bodytext = ['paper_id', 'mag_field_of_study', 'body_text', 'bodytext_p_parsed3']
    df_abstract1=df_abstract[list_col_abstract]
    df_bodytext1=df_bodytext[list_col_bodytext]
    df_abstract1=df_abstract1.rename(columns={"abstract_x_p_parsed3": "abstract_parsed"})
    df_bodytext1=df_bodytext1.rename(columns={"bodytext_p_parsed3": "bodytext_parsed"})
    
    fieldstudy_codes={
    'Medicine': 0,
    'Chemistry': 1,
    'Physics': 2,
    'Mathematics': 3,
    'Biology': 4,
    'Computer Science':5
    }
    df_abstract1["fieldstudy_code"]= df_abstract1["mag_field_of_study"]
    df_abstract1 = df_abstract1.replace({"fieldstudy_code":fieldstudy_codes})
    df_bodytext1["fieldstudy_code"]= df_bodytext1["mag_field_of_study"]
    df_bodytext1 = df_bodytext1.replace({"fieldstudy_code":fieldstudy_codes})
    
    df_abstract1.to_feather(f'20200705v1/full/text_processed/abstract_{file_index}.feather')
    df_bodytext1.to_feather(f'20200705v1/full/text_processed/bodytext_{file_index}.feather')
    

In [ ]:
for path in tqdm(all_files):
    process_textfile(path)

  6%|▌         | 2/35 [4:22:03<73:31:59, 8021.80s/it]